In [2]:
from calculating_risk_for_single_community import calculating_risk_for_single_community as single_calculate
import numpy as np
import pandas as pd
import math
import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
'''
Method
1) Daily new cases
2) Model outputs
    a) Aggregate RPI data to communities
    b) Unify the format
3) Convert to Value between -1,0,1
4) Calculate correlation coefficient
'''


'\nMethod\n1) Daily new cases\n2) Model outputs\n    a) Aggregate RPI data to communities\n    b) Unify the format\n3) Convert to Value between -1,0,1\n4) Calculate correlation coefficient\n'

In [7]:
# input data into dataframes
population = pd.read_csv('processed_population.csv')

url_infe = "https://raw.githubusercontent.com/ANRGUSC/covid19_risk_estimation/master/data/Covid-19.csv"
infection = pd.read_csv(url_infe)

url_usc = "https://raw.githubusercontent.com/ANRGUSC/covid19_risk_estimation/master/data/Covid-19-R.csv"
risk_usc = pd.read_csv(url_usc)

url_rpi = "https://raw.githubusercontent.com/Yueyang-Li-Elfa/Risk-Score-RPI-Solver/master/Latest%20Risk%20Score/risk_score.csv"
risk_rpi = pd.read_csv(url_rpi)

url_umich1 = "https://raw.githubusercontent.com/skasralikar/Risk-Score-1-UMichZJU/master/data/output/LA_daily_out.csv"
risk_umich1 = pd.read_csv(url_umich1,error_bad_lines=False)

# url_lmu = "https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/final_risk_score.csv"
# risk_lmu = pd.read_csv(url_lmu)

url_umich3 = "https://raw.githubusercontent.com/sli525/rmds-lab-team3-project/master/output%20file/risk_level.csv"
risk_umich3 = pd.read_csv(url_umich3)


In [8]:
# Change all regions/locations to Title Style
infection['Region'] = infection.apply(lambda x: x['Region'].replace('UNINCORPORATED - ','').title().replace('city of ',''), axis = 1)

print(population.head(),'\n',infection.head(),'\n',risk_usc.head(),'\n',risk_umich1.head(),'\n',risk_umich3.head())

            Region   Latitude   Longitude  Population
0       Northridge  34.234561 -118.536932       67755
1  Exposition Park  34.013654 -118.287211       45190
2          Norwalk  33.909280 -118.084917      104868
3        Hawthorne  33.918859 -118.348326      114032
4       El Segundo  33.917028 -118.415634       26864 
    Time Stamp         Region   Latitude   Longitude  Number of cases
0  03-16-2020       Alhambra  34.093042 -118.127060                2
1  03-16-2020        Arcadia  34.136208 -118.040150                1
2  03-16-2020  Beverly Hills  34.069650 -118.396306                1
3  03-16-2020  Boyle Heights  34.043689 -118.209768                5
4  03-16-2020         Carson  33.832204 -118.251755                1 
    Unnamed: 0  Time Stamp         Region   Latitude   Longitude  Risk-Score  \
0           0  03-16-2020       Alhambra  34.093042 -118.127060    0.158705   
1           1  03-16-2020        Arcadia  34.136208 -118.040150    0.142132   
2           2  03-16-

In [39]:
# Create a variable to save the latest start date and earlist end date
date_inter=["01-01-2020","12-31-2020"]
date_inter[0] = datetime.datetime.strptime(date_inter[0],'%m-%d-%Y')
date_inter[1] = datetime.datetime.strptime(date_inter[1],'%m-%d-%Y')

In [40]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# USC
la_regions = risk_usc.Region.unique()

model_usc = pd.DataFrame()
model_usc["Region"] = la_regions
model_usc["cases"] = np.empty((len(model_usc), 0)).tolist()
model_usc["risk"] = np.empty((len(model_usc), 0)).tolist()
model_usc["first_date"] = np.empty((len(model_usc), 0)).tolist()
model_usc["last_date"] = np.empty((len(model_usc), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_usc.sort_values(by="Time Stamp")

for index, row in model_usc.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_usc.loc[risk_usc['Region'] == row["Region"]]
    model_usc["cases"][index]= case_r["Number of cases"].tolist()
    model_usc["risk"][index]= risk_r["Risk-Score"].tolist()
    
    # Get min and max Time stamp
    model_usc["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_usc['Time Stamp'])]
    model_usc["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_usc['Time Stamp'])]

# Update the time interval
usc_first = datetime.datetime.strptime(model_usc["first_date"][0][1],'%m-%d-%Y')
usc_last = datetime.datetime.strptime(model_usc["last_date"][0][1],'%m-%d-%Y')
if usc_first>date_inter[0]:
    date_inter[0]=usc_first
if usc_last<date_inter[1]:
    date_inter[1]=usc_last
print(model_usc)

                       Region  \
0                    Alhambra   
1                     Arcadia   
2               Beverly Hills   
3               Boyle Heights   
4                      Carson   
..                        ...   
237            South Pasadena   
238             Victoria Park   
239         Wellington Square   
240  West Whittier/Los Nietos   
241                Wilmington   

                                                 cases  \
0    [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
1    [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
2    [1, 1, 4, 4, 5, 7, 8, 12, 12, 13, 15, 21, 21, ...   
3    [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 8, 8, 10, 11...   
4    [1, 1, 1, 1, 3, 3, 4, 6, 8, 9, 21, 26, 32, 35,...   
..                                                 ...   
237  [3, 5, 5, 6, 7, 12, 15, 15, 17, 17, 17, 20, 21...   
238  [0, 0, 0, 0, 0, 17, 17, 17, 17, 18, 18, 18, 18...   
239  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, ...   
240  [0, 0, 0, 0, 1, 10

In [42]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# umich1
la_regions = risk_umich1.Region.unique()

model_umich1 = pd.DataFrame()
model_umich1["Region"] = la_regions
model_umich1["cases"] = np.empty((len(model_umich1), 0)).tolist()
model_umich1["risk"] = np.empty((len(model_umich1), 0)).tolist()
model_umich1["first_date"] = np.empty((len(model_umich1), 0)).tolist()
model_umich1["last_date"] = np.empty((len(model_umich1), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_umich1.sort_values(by="Timestamp")

for index, row in model_umich1.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_umich1.loc[risk_umich1['Region'] == row["Region"]]
    model_umich1["cases"][index]= case_r["Number of cases"].tolist()
    model_umich1["risk"][index]= risk_r["Risk_score"].tolist()
    
    # Get min and max Time stamp
    model_umich1["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_umich1['Timestamp']).strip()]
    model_umich1["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_umich1['Timestamp']).strip()]

# Update the time interval
umich1_first = datetime.datetime.strptime(model_umich1["first_date"][0][1],'%Y-%m-%d')
umich1_last = datetime.datetime.strptime(model_umich1["last_date"][0][1],'%Y-%m-%d')
if umich1_first>date_inter[0]:
    date_inter[0]=umich1_first
if umich1_last<date_inter[1]:
    date_inter[1]=umich1_last

    
print(model_umich1)

              Region                                              cases  \
0              Acton  [0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 5, 5, ...   
1       Agoura Hills  [1, 3, 3, 5, 6, 9, 9, 9, 11, 10, 20, 20, 20, 2...   
2           Alhambra  [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
3           Altadena  [1, 2, 3, 3, 5, 5, 5, 6, 9, 10, 10, 10, 12, 14...   
4            Arcadia  [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
..               ...                                                ...   
152      Willowbrook  [1, 1, 1, 1, 1, 1, 2, 2, 5, 25, 27, 27, 27, 29...   
153       Wilmington  [7, 10, 10, 12, 13, 28, 32, 33, 33, 34, 40, 42...   
154  Wilshire Center  [1, 1, 2, 2, 4, 4, 5, 8, 8, 12, 40, 44, 46, 47...   
155         Winnetka  [1, 1, 1, 1, 2, 3, 5, 3, 8, 9, 9, 10, 11, 43, ...   
156   Woodland Hills  [2, 3, 3, 4, 5, 5, 7, 8, 11, 13, 14, 18, 19, 2...   

                                                  risk  \
0    [-0.3658003399102317, -2.60065757800

In [44]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# umich3
la_regions = risk_umich3.Region.unique()

model_umich3 = pd.DataFrame()
model_umich3["Region"] = la_regions
model_umich3["cases"] = np.empty((len(model_umich3), 0)).tolist()
model_umich3["risk"] = np.empty((len(model_umich3), 0)).tolist()
model_umich3["first_date"] = np.empty((len(model_umich3), 0)).tolist()
model_umich3["last_date"] = np.empty((len(model_umich3), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_umich3.sort_values(by="Time Stamp")

for index, row in model_umich3.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_umich3.loc[risk_umich3['Region'] == row["Region"]]
    model_umich3["cases"][index]= case_r["Number of cases"].tolist()
    model_umich3["risk"][index]= risk_r["risk_score"].tolist()
    
    # Get min and max Time stamp
    model_umich3["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_umich3['Time Stamp']).strip()]
    model_umich3["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_umich3['Time Stamp']).strip()]


# Update the time interval
umich3_first = datetime.datetime.strptime(model_umich3["first_date"][0][1],'%Y-%m-%d')
umich3_last = datetime.datetime.strptime(model_umich3["last_date"][0][1],'%Y-%m-%d')
if umich3_first>date_inter[0]:
    date_inter[0]=umich3_first
if umich3_last<date_inter[1]:
    date_inter[1]=umich3_last    
    
print(model_umich3)

[datetime.datetime(2020, 4, 7, 0, 0), datetime.datetime(2020, 7, 31, 0, 0)]
               Region                                              cases  \
0        Agoura Hills  [1, 3, 3, 5, 6, 9, 9, 9, 11, 10, 20, 20, 20, 2...   
1            Alhambra  [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
2            Altadena  [1, 2, 3, 3, 5, 5, 5, 6, 9, 10, 10, 10, 12, 14...   
3             Arcadia  [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
4              Arleta  [1, 2, 2, 4, 5, 5, 7, 7, 23, 24, 24, 26, 29, 3...   
..                ...                                                ...   
151             Acton  [0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 5, 5, ...   
152  Lake Los Angeles  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 8, 8, ...   
153    South El Monte  [1, 1, 1, 1, 1, 1, 3, 4, 0, 0, 0, 0, 0, 5, 7, ...   
154           Bassett  [0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 13, 15, 15, 20,...   
155            Avalon  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

           

In [55]:
print(len(model_umich3['risk'][1]))

112


In [ ]:
# We don't have long time series of RPI data so will not be included in the model this time
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# RPI

la_regions = risk_rpi["community"].unique()
print(la_regions)
print(risk_rpi.dropna())
risk_rpi = risk_rpi.dropna(thresh=26)
comm_rpi = risk_rpi.groupby(["community","weekday"])["risk_score"].apply(lambda x : x.astype(float).mean()).reset_index()

la_regions = comm_rpi["community"].unique()
print(la_regions)


model_rpi = pd.DataFrame()
model_rpi["Region"] = la_regions
model_rpi["cases"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["risk"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["first_date"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["last_date"] = np.empty((len(model_rpi), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_rpi.sort_values(by="Time Stamp")

for index, row in model_rpi.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_rpi.loc[risk_rpi['Region'] == row["Region"]]
    model_rpi["cases"][index]= case_r["Number of cases"].tolist()
    model_rpi["risk"][index]= risk_r["risk_score"].tolist()
    
    # Get min and max Time stamp
    model_rpi["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_rpi['Time Stamp'])]
    model_rpi["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_rpi['Time Stamp'])]
    
print(model_rpi)

In [56]:
# Choose intersected data based on dates
def column_slice(df,data_column,last_date_column,case_risk,first_d,last_d):
    case_len = (first_d - last_d).days
    format_date = ["%m-%d-%Y","%Y-%m-%d"]
    case_l_delta = (last_d - datetime.datetime.strptime(df[last_date_column][0][case_risk],format_date[case_risk])).days
    case_m = df[data_column].tolist()

    for i in range(len(case_m)):
        j = len(case_m[i])
        if case_l_delta < 0:
            case_m[i] = case_m[i][(case_l_delta+case_len):case_l_delta]
        elif case_l_delta == 0:
            case_m[i] = case_m[i][case_len:]

    df[data_column] = case_m

In [93]:
# Ensemble

# merge two or more dataframes into one

df_ensemble = model_usc.merge(model_umich1,on='Region').merge(model_umich3,on='Region')
df_ensemble = df_ensemble[["Region","cases","risk_x","risk_y","risk","first_date_x","last_date_x","first_date_y","last_date_y","first_date","last_date"]]
df_ensemble = df_ensemble.rename(columns={'risk_x': 'risk_usc','risk_y':'risk_umich1','risk':'risk_umich3','first_date_x':'first_date_usc','first_date_y':'first_date_umich1','first_date':'first_date_umich3','last_date_x':'last_date_usc','last_date_y':'last_date_umich1','last_date':'last_date_umich3'})

'''
Use the intersection generated from all the data

'''
df_merged = df_ensemble
first_d = date_inter[0]
last_d = date_inter[1]

# Temperarily change first day to July 12th, and define the last day to be 3-day less
first_d = datetime.datetime.strptime('2020-7-12','%Y-%m-%d')
last_d = last_d - datetime.timedelta(days=3)
# cases portion
column_slice(df_ensemble,"cases","last_date_usc",0,first_d,last_d)
# usc portion
column_slice(df_ensemble,"risk_usc","last_date_usc",0,first_d,last_d)
# umich1 portion
column_slice(df_ensemble,"risk_umich1","last_date_umich1",1,first_d,last_d)
# umich3 portion
column_slice(df_ensemble,"risk_umich3","last_date_umich3",1,first_d,last_d)

print(len(df_ensemble['risk_usc'][100]),len(df_ensemble['cases'][100]),len(df_ensemble['risk_umich1'][0]),len(df_ensemble['risk_umich3'][0]))


df_ensemble = df_ensemble[['Region','cases','risk_usc','risk_umich1','risk_umich3']]
df_ensemble["first_last"] = [date_inter for i in df_ensemble.index]
df_ensemble

16 16 16 16


,Region,cases,risk_usc,risk_umich1,risk_umich3,first_last
0,Alhambra,"[776, 801, 812, 820, 848, 881, 488, 892, 905, ...","[6.334667101824095, 7.876583102858233, 8.89987...","[1.906676763981268, 1.6991966573197532, 2.0950...","[1.868136167, 2.189863707, 1.854310047, 2.0217...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
1,Arcadia,"[284, 291, 292, 299, 301, 306, 182, 309, 312, ...","[2.043488333509353, 2.210074923028474, 2.03455...","[1.1669964257323306, 1.0014136846908728, 0.786...","[-0.03379541484, 0.2638715443, -0.4707836469, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
2,Beverly Hills,"[522, 527, 530, 530, 530, 530, 264, 534, 538, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.6185010382120333, ...","[4.231510648930675, 4.589190248121036, 3.99712...","[5.064603477, 3.965281338, 4.84351195, 4.25783...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
3,Boyle Heights,"[2839, 2916, 2932, 2980, 3005, 3108, 1866, 316...","[19.186921824521587, 20.81939194752828, 22.769...","[6.194442254851607, 6.841771984795374, 6.62323...","[6.6983975860000005, 6.993530772000001, 6.3579...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
4,Carson,"[1413, 1452, 1465, 1478, 1492, 1521, 912, 1566...","[1.062231743806238, 1.4706756755605832, 1.8591...","[2.4450561317599164, 2.4766777905974924, 2.901...","[2.9098556660000003, 2.892181557, 2.71029983, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
...,...,...,...,...,...,...
151,Huntington Park,"[1838, 1881, 1889, 1917, 1943, 1997, 1263, 203...","[10.993702373892361, 11.347559388885431, 12.15...","[5.5728929136443135, 5.630830065792656, 5.1752...","[5.947917996, 6.101854115, 5.541580322000001, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
152,Koreatown,"[739, 755, 764, 781, 788, 818, 513, 830, 839, ...","[4.362945184378066, 5.785414396362393, 7.15624...","[1.0140748776931212, 0.8327492087123768, 1.167...","[0.9200207269, 0.8762491726, 0.863524287999999...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
153,Mt. Washington,"[321, 335, 336, 349, 352, 373, 200, 374, 380, ...","[4.518748344852191, 5.00651582242222, 5.380264...","[2.632367393642308, 2.7763695555014998, 4.1388...","[1.397697544, 2.755354704, 3.027869874, 3.9339...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"
154,South Pasadena,"[206, 208, 208, 208, 209, 211, 168, 217, 218, ...","[0.25898438810606844, 0.39697293722254456, 0.6...","[0.7474239493615195, 0.9726646117887806, 1.087...","[0.1022657686, 0.8941884738, 1.283249865, 1.34...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]"


In [94]:
def n_case(n_ahead, case):
    n = len(case)
    new_infe = []
    for i in range(n):
        if i<n_ahead:
            new_infe.append(0)
        else:
            new_infe.append(case[i]-case[i-n_ahead])
    new_infe = (np.array(new_infe)-np.min(new_infe)) / (np.max(new_infe)-np.min(new_infe))
    new_infe = np.round(new_infe,6)
    
    return new_infe.tolist()

def n_risk(risk):
    n = len(risk)

    risk = (np.array(risk)-np.min(risk)) / (np.max(risk)-np.min(risk))
    risk = np.round(risk,6)
    return risk.tolist()

In [95]:
df_ensemble['3-case'] = 0
df_ensemble['3-case'] = df_ensemble['3-case'].astype('object')
df_ensemble['7-case'] = 0
df_ensemble['7-case'] = df_ensemble['7-case'].astype('object')
df_ensemble['n-usc'] = 0
df_ensemble['n-usc'] = df_ensemble['n-usc'].astype('object')
df_ensemble['n-umich1'] = 0
df_ensemble['n-umich1'] = df_ensemble['n-umich1'].astype('object')
df_ensemble['n-umich3'] = 0
df_ensemble['n-umich3'] = df_ensemble['n-umich3'].astype('object')

for index, row in df_ensemble.iterrows():
    case = np.array(row['cases'], dtype = 'float')
    risk_usc = np.array(row['risk_usc'], dtype = 'float')
    risk_umich1 = np.array(row['risk_umich1'], dtype = 'float')
    risk_umich3 = np.array(row['risk_umich3'], dtype = 'float')
    
    if sum(risk_usc)<=0:
        continue
    df_ensemble.set_value(index, '3-case', n_case(3, case))
    #df_ensemble.set_value(index, '7-case', n_case(7, case))
    df_ensemble.set_value(index, 'n-usc', n_risk(risk_usc))
    df_ensemble.set_value(index, 'n-umich1', n_risk(risk_umich1))
    df_ensemble.set_value(index, 'n-umich3', n_risk(risk_umich3))

In [96]:
df_ensemble

,Region,cases,risk_usc,risk_umich1,risk_umich3,first_last,3-case,7-case,n-usc,n-umich1,n-umich3
0,Alhambra,"[776, 801, 812, 820, 848, 881, 488, 892, 905, ...","[6.334667101824095, 7.876583102858233, 8.89987...","[1.906676763981268, 1.6991966573197532, 2.0950...","[1.868136167, 2.189863707, 1.854310047, 2.0217...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.509235, 0.509235, 0.509235, 0.567282, 0.571...",0,"[0.696777, 0.866379, 0.978936, 1.0, 0.8698, 0....","[0.736746, 0.446756, 1.0, 0.721439, 0.826853, ...","[0.651729, 1.0, 0.636763, 0.817966, 0.638781, ..."
1,Arcadia,"[284, 291, 292, 299, 301, 306, 182, 309, 312, ...","[2.043488333509353, 2.210074923028474, 2.03455...","[1.1669964257323306, 1.0014136846908728, 0.786...","[-0.03379541484, 0.2638715443, -0.4707836469, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.5, 0.5, 0.5, 0.561983, 0.541322, 0.557851, ...",0,"[0.228254, 0.292661, 0.224799, 0.114825, 0.088...","[1.0, 0.828007, 0.604937, 0.591502, 0.0, 0.482...","[0.416917, 0.64881, 0.076487, 0.702539, 0.0, 0..."
2,Beverly Hills,"[522, 527, 530, 530, 530, 530, 264, 534, 538, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.6185010382120333, ...","[4.231510648930675, 4.589190248121036, 3.99712...","[5.064603477, 3.965281338, 4.84351195, 4.25783...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.537374, 0.537374, 0.537374, 0.553535, 0.543...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.071349, 1.0, 0.115...","[0.898214, 1.0, 0.831515, 0.90163, 0.608078, 0...","[1.0, 0.733288, 0.94636, 0.804267, 0.90408, 0...."
3,Boyle Heights,"[2839, 2916, 2932, 2980, 3005, 3108, 1866, 316...","[19.186921824521587, 20.81939194752828, 22.769...","[6.194442254851607, 6.841771984795374, 6.62323...","[6.6983975860000005, 6.993530772000001, 6.3579...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.50427, 0.50427, 0.50427, 0.56161, 0.540464,...",0,"[0.698605, 0.782274, 0.88224, 0.979957, 1.0, 0...","[0.52156, 0.752644, 0.674632, 0.617572, 0.2445...","[0.88634, 0.991047, 0.765571, 1.0, 0.756899, 0..."
4,Carson,"[1413, 1452, 1465, 1478, 1492, 1521, 912, 1566...","[1.062231743806238, 1.4706756755605832, 1.8591...","[2.4450561317599164, 2.4766777905974924, 2.901...","[2.9098556660000003, 2.892181557, 2.71029983, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.496448, 0.496448, 0.496448, 0.547751, 0.528...",0,"[0.0, 0.204296, 0.398596, 0.632241, 0.869554, ...","[0.609199, 0.636271, 1.0, 0.712674, 0.607633, ...","[1.0, 0.9888, 0.873547, 0.872474, 0.797918, 0...."
...,...,...,...,...,...,...,...,...,...,...,...
151,Huntington Park,"[1838, 1881, 1889, 1917, 1943, 1997, 1263, 203...","[10.993702373892361, 11.347559388885431, 12.15...","[5.5728929136443135, 5.630830065792656, 5.1752...","[5.947917996, 6.101854115, 5.541580322000001, ...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.509343, 0.509343, 0.509343, 0.564014, 0.552...",0,"[0.773245, 0.811654, 0.898941, 0.997427, 1.0, ...","[0.620361, 0.649193, 0.422482, 0.333248, 0.191...","[0.955556, 1.0, 0.838238, 0.750777, 0.641215, ..."
152,Koreatown,"[739, 755, 764, 781, 788, 818, 513, 830, 839, ...","[4.362945184378066, 5.785414396362393, 7.15624...","[1.0140748776931212, 0.8327492087123768, 1.167...","[0.9200207269, 0.8762491726, 0.863524287999999...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.506557, 0.506557, 0.506557, 0.57541, 0.5606...",0,"[0.560951, 0.743841, 0.92009, 1.0, 0.906605, 0...","[0.470478, 0.257134, 0.651015, 0.324291, 0.0, ...","[0.558786, 0.467096, 0.44044, 0.623434, 0.5960..."
153,Mt. Washington,"[321, 335, 336, 349, 352, 373, 200, 374, 380, ...","[4.518748344852191, 5.00651582242222, 5.380264...","[2.632367393642308, 2.7763695555014998, 4.1388...","[1.397697544, 2.755354704, 3.027869874, 3.9339...","[2020-04-07 00:00:00, 2020-07-31 00:00:00]","[0.495575, 0.495575, 0.495575, 0.578171, 0.545...",0,"[0.824959, 0.924062, 1.0, 0.993935, 0.844843, ...","[0.206512, 0.278997, 0.964827, 1.0, 0.355339, ...","[0.0, 0.456104, 0.547655, 0.852063, 0.621807, ..."
154,South Pasadena,"[206, 208, 208, 2

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

def lin_reg(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=9)
    lin_reg_mod = LinearRegression()
    lin_reg_mod.fit(X_train, y_train)
    pred = lin_reg_mod.predict(X_test)
    # RMSE
    test_set_rmse = (np.sqrt(mean_squared_error(y_test, pred)))
    # r square
    test_set_r2 = lin_reg_mod.score(X_test,y_test)

    output = np.array([lin_reg_mod.coef_,lin_reg_mod.intercept_,test_set_rmse])
    return output

In [98]:
df_ensemble = df_ensemble[df_ensemble["3-case"] != 0]
df_ensemble.to_csv("ensemble_pre.csv")

In [99]:
df_ensemble['ensemble'] = 0
df_ensemble['ensemble'] = df_ensemble['ensemble'].astype('object')

# Ensemble to do 3-day prediction
for index, row in df_ensemble.iterrows():
    try:


        y = np.array(row['3-case'], dtype = 'object')
        x_usc = np.array(row['n-usc'], dtype = 'object')
        x_umich = np.array(row['n-umich1'], dtype = 'object')
        x_rmds3 = np.array(row['n-umich3'], dtype = 'object')
        X = pd.DataFrame({'n_usc': x_usc, 'n_umich1': x_umich,'n_umich3': x_rmds3})
        df_ensemble.set_value(index, 'ensemble', lin_reg(X,y))
    except:
        pass

In [100]:
df_ensemble.to_csv("ensemble_result.csv")

PermissionError: [Errno 13] Permission denied: 'ensemble_result.csv'

In [101]:
# Use the last 3 days data to do prediction and evaluation

last_d = date_inter[1]
first_d = last_d - datetime.timedelta(days=3)
df_test=df_merged

# cases portion
column_slice(df_test,"cases","last_date_usc",0,first_d,last_d)
# usc portion
column_slice(df_test,"risk_usc","last_date_usc",0,first_d,last_d)
# umich1 portion
column_slice(df_test,"risk_umich1","last_date_umich1",1,first_d,last_d)
# umich3 portion
column_slice(df_test,"risk_umich3","last_date_umich3",1,first_d,last_d)






              Region               cases  \
0           Alhambra     [848, 881, 488]   
1            Arcadia     [301, 306, 182]   
2      Beverly Hills     [530, 530, 264]   
3      Boyle Heights  [3005, 3108, 1866]   
4             Carson   [1492, 1521, 912]   
..               ...                 ...   
151  Huntington Park  [1943, 1997, 1263]   
152        Koreatown     [788, 818, 513]   
153   Mt. Washington     [352, 373, 200]   
154   South Pasadena     [209, 211, 168]   
155       Wilmington   [1100, 1122, 637]   

                                              risk_usc  \
0    [7.907686765831624, 5.595306736688002, 5.51189...   
1    [1.6813146829751842, 2.13373768947335, 1.93541...   
2           [0.0, 0.6185010382120333, 8.6686830350422]   
3    [25.067465161390988, 22.82635673830817, 16.541...   
4    [2.8007098108694137, 3.0615081835031597, 1.226...   
..                                                 ...   
151  [13.082746499791506, 11.53415047935874, 10.527...   
152  [7

In [102]:
df_test = df_test[['Region','cases','risk_usc','risk_umich1','risk_umich3']]
df_ensemble = df_ensemble[['Region','ensemble']]
result = pd.concat([df_test, df_ensemble], axis=1, join='inner')
result

,Region,cases,risk_usc,risk_umich1,risk_umich3,Region,ensemble
0,Alhambra,"[848, 881, 488]","[7.907686765831624, 5.595306736688002, 5.51189...","[1.7618800444941396, 1.621834334060627, 1.7204...","[1.42001308, 1.734452242, 1.389876945]",Alhambra,"[[-0.49711322001686203, 0.1359152061122531, 0...."
1,Arcadia,"[301, 306, 182]","[1.6813146829751842, 2.13373768947335, 1.93541...","[0.6684681342228085, 0.615143978107874, 0.7221...","[0.4600938712, -0.2332385999, 0.007315162696]",Arcadia,"[[0.3260456188751174, -0.04091768074470035, 0...."
2,Beverly Hills,"[530, 530, 264]","[0.0, 0.6185010382120333, 8.6686830350422]","[2.6973272293895048, 2.9660897862601474, 3.300...","[1.238973167, 0.9759429035, 1.304550575]",Beverly Hills,"[[-0.7463676171923735, -0.10738262394375653, -..."
3,Boyle Heights,"[3005, 3108, 1866]","[25.067465161390988, 22.82635673830817, 16.541...","[6.173115466708143, 5.217123691690935, 4.73340...","[5.794096859, 5.78407894, 4.9214044580000005]",Boyle Heights,"[[0.05050682474565402, 0.7114159198107027, -0...."
4,Carson,"[1492, 1521, 912]","[2.8007098108694137, 3.0615081835031597, 1.226...","[2.282495390775276, 2.0983951077309317, 2.1383...","[1.991131481, 2.349739606, 1.980709424]",Carson,"[[0.290235347608934, -0.15117099909172463, 0.3..."
...,...,...,...,...,...,...,...
151,Huntington Park,"[1943, 1997, 1263]","[13.082746499791506, 11.53415047935874, 10.527...","[5.107228212448252, 4.32628502922686, 4.674836...","[5.187990146000001, 4.7871007080000005, 4.2412...",Huntington Park,"[[-0.09630971109365778, 0.6088728998012476, -0..."
152,Koreatown,"[788, 818, 513]","[7.051360532003738, 5.007945541637974, 2.07179...","[0.7527397708059073, 0.8950785010926179, 1.117...","[1.130648572, 1.044372913, 0.9002968989]",Koreatown,"[[-0.2734083360343371, -0.19138303274896806, -..."
153,Mt. Washington,"[352, 373, 200]","[4.6166118588936955, 3.3650238912757167, 1.330...","[2.8833458440098503, 3.5663568171819446, 4.197...","[3.425000525, 3.484671855, 2.935706707]",Mt. Washington,"[[-0.04672510515269588, 0.3826026716226048, -0..."
154,South Pasadena,"[209, 211, 168]","[1.5633260569666791, 1.6878879059294087, 0.687...","[0.7290124589070633, 0.5679714656663923, 0.464...","[0.4346696005, -0.07880903667, 0.1430850664000...",South Pasadena,"[[0.05995447726161943, 0.5042462031616063, -0...."


In [ ]:

result['predict'] = 0
result['predict'] = result['predict'].astype('object')
for index, row in df_ensemble.iterrows():
    try:
        result.set_value(index, 'predict', row["n-usc"][-1]*row["ensemble"][0][0]+row["n-umich1"][-1]*row["ensemble"][0][1]+row["n-umich3"][-1]*row["ensemble"][0][2]+row["ensemble"][1])
    except:
        pass
df_ensemble.to_csv("ensemble_prediction.csv")